---
execute:
  cache: false
  eval: true
  echo: true
  warning: false
jupyter: python3
---

# Hyperparameter Tuning with PyTorch Lightning and User Data Sets  {#sec-light-user-data-1001}

In [1]:
#| echo: false
#| label: 1001_user_data_imports
import numpy as np
import os
from math import inf
import numpy as np
import warnings
if not os.path.exists('./figures'):
    os.makedirs('./figures')
warnings.filterwarnings("ignore")

In this section, we will show how user specfied data can be used for the `PyTorch` Lightning hyperparameter tuning workflow with `spotpython`.

## Loading a User Specified Data Set

Using a user-specified data set is straightforward.

The user simply needs to provide a data set and loads is as a  `spotpython`  `CVSDataset()` class by specifying the path, filename, and target column.

Consider the following example, where the user has a data set stored in the `userData` directory. The data set is stored in a file named `data.csv`. The target column is named `target`. To show the data, it is loaded as a `pandas` data frame and the first 5 rows are displayed. This step is not necessary for the hyperparameter tuning process, but it is useful for understanding the data.

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd

# from utils import *
# from train import *

import pickle
import matplotlib.pyplot as plt
from torch.utils.data import Dataset

In [3]:
device = 'cpu'
epochs = 2000
seeds = [42, 43, 44, 45, 46, 47, 48, 49, 50, 51]
input_features = ['PI tot V [-]', 'Bereich u2red']
data = pd.read_csv("/Users/bartz/workspace/schu25a_netgen_gecco/data/data_man_tca88.csv")
data["source_file"] = ["tca88"] * len(data)

In [4]:
class ManyToManyDataset(Dataset):
    def __init__(
        self,
        df_list,
        target,
        drop=None,
        dtype=torch.float32,
    ):
        try: 
            self.data = [df.drop(drop, axis=1) for df in df_list]
        except:
            self.data = df_list
        self.target = [torch.tensor(df[target].to_numpy(), dtype=dtype) for df in self.data]
        self.features = [torch.tensor(df.drop([target], axis=1).to_numpy(), dtype=dtype)for df in self.data]

    def __getitem__(self, index):
        x = self.features[index]
        y = self.target[index]
        return x, y

    def __len__(self):
        return len(self.data)
    
    
class ManyToOneDataset(Dataset):
    def __init__(
        self,
        df_list,
        target,
        drop=None,
        dtype=torch.float32,
    ):
        try: 
            self.data = [df.drop(drop, axis=1) for df in df_list]
        except:
            self.data = df_list
        self.target = [torch.tensor(df[target].to_numpy()[0], dtype=dtype) for df in self.data]
        self.features = [torch.tensor(df.drop([target], axis=1).to_numpy(), dtype=dtype)for df in self.data]

    def __getitem__(self, index):
        x = self.features[index]
        y = self.target[index]
        return x, y

    def __len__(self):
        return len(self.data)

In [5]:
def load_data(data,
              input_features=['V tot V red [m³/s]'], 
              target= 'PI tot V [-]' ,
              drop = ['Bereich u2red','source_file'],
              group_by='Bereich u2red',
              feature_scaling=None, 
              target_scaling=None, 
              create_dataset=True, 
              dataset_type='many_to_many'):

    if feature_scaling is not None:
        data[input_features] = feature_scaling.fit_transform(data[input_features])
    
    if target_scaling is not None:
        data[target] = target_scaling.fit_transform(data[target])    
    
    if create_dataset == False:
        return data
    else:
        groups = []
        groups_name = []
        data_groups = data.groupby(group_by)

    for name, group in data_groups:
        groups.append(group)
        groups_name.append(name)
    
    if dataset_type == 'many_to_many':
        return ManyToManyDataset(groups, target=target, drop=drop), data
    elif dataset_type == "many_to_one":
        return ManyToOneDataset(groups, target=target, drop=drop), data

In [6]:
kennlinienfelder = data.groupby("source_file")
pred_dict = {}

seeds = [42]
for kennlinienfeld in kennlinienfelder:
    print(f"kennlinienfeld: {kennlinienfeld[0]}")
    data_name = kennlinienfeld[0]
    print(data_name)
    ds, data = ds, _ = load_data(kennlinienfeld[1], 
                                input_features=['PI tot V [-]' ], 
                                target='V tot V red [m³/s]',
                                drop = ['source_file', "Bereich u2red"],
                                group_by="Bereich u2red",
                                # feature_scaling=MinMaxScaler()
                                )
    
    pred_dict[data_name] = {}

    for seed in seeds:
        print(f"seed: {seed}")
        # seed_everything(seed)
        g = torch.Generator()
        g.manual_seed(seed)
        
        pred_dict[data_name][seed] = {}
        pred_dict[data_name][seed]['x'] = []
        pred_dict[data_name][seed]['y_hat'] = []
        pred_dict[data_name][seed]['y'] = []
        pred_dict[data_name][seed]['mape'] = []
        pred_dict[data_name][seed]['rmse'] = []
        
        # Create indices for the split
        indices = list(range(len(ds)))        
        for i in indices:
            print(f"indices: {indices}")
            test_indices = [indices[i]]
            train_indices = [index for index in indices if index != test_indices[0]]
            
            train_dataset = torch.utils.data.Subset(ds, train_indices)
            test_dataset = torch.utils.data.Subset(ds, test_indices)

kennlinienfeld: tca88
tca88
seed: 42
indices: [0, 1, 2, 3, 4, 5, 6, 7, 8]
indices: [0, 1, 2, 3, 4, 5, 6, 7, 8]
indices: [0, 1, 2, 3, 4, 5, 6, 7, 8]
indices: [0, 1, 2, 3, 4, 5, 6, 7, 8]
indices: [0, 1, 2, 3, 4, 5, 6, 7, 8]
indices: [0, 1, 2, 3, 4, 5, 6, 7, 8]
indices: [0, 1, 2, 3, 4, 5, 6, 7, 8]
indices: [0, 1, 2, 3, 4, 5, 6, 7, 8]
indices: [0, 1, 2, 3, 4, 5, 6, 7, 8]


The following step is not necessary for the hyperparameter tuning process, but it is useful for understanding the data. The data set is loaded as a `DataLoader` from `torch.utils.data` to check the data.

In [7]:
#| label: 1001_user_data_dataloader
# Set batch size for DataLoader
batch_size = 1
# Create DataLoader
from torch.utils.data import DataLoader
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

# Iterate over the data in the DataLoader
for batch in dataloader:
    inputs, targets = batch
    print(f"Batch Size: {inputs.size(0)}")
    print(f"Inputs Shape: {inputs.shape}")
    print(f"Targets Shape: {targets.shape}")
    print("---------------")
    print(f"Inputs: {inputs}")
    print(f"Targets: {targets}")
    break

Batch Size: 1
Inputs Shape: torch.Size([1, 6, 1])
Targets Shape: torch.Size([1, 6])
---------------
Inputs: tensor([[[1.2800],
         [1.4400],
         [1.4700],
         [1.4900],
         [1.5100],
         [1.5400]]])
Targets: tensor([[2.0100, 1.7700, 1.6800, 1.5900, 1.4500, 1.0400]])


In [8]:
from spotpython.data.lightdatamodule import LightDataModule
data_module = LightDataModule(dataset=train_dataset, batch_size=5, test_size=0.5)
data_module.setup()
print(f"Training set size: {len(data_module.data_train)}")

Training set size: 2


In [9]:
len(train_dataset), len(test_dataset)

(8, 1)

In [10]:
from spotpython.hyperdict.light_hyper_dict import LightHyperDict
from spotpython.fun.hyperlight import HyperLight
from spotpython.utils.init import (fun_control_init, surrogate_control_init, design_control_init)
from spotpython.utils.eda import print_res_table
from spotpython.hyperparameters.values import set_hyperparameter
from spotpython.spot import Spot
from math import inf


fun_control=fun_control_init(
    penalty_NA=1000,
    ocba_delta=1,
    TENSORBOARD_CLEAN=True,
    tensorboard_log=True,
    accelerator="cpu",
    collate_fn_name="PadSequenceManyToMany",
    show_config=True,
    verbosity=1,
    save_experiment=False,
    save_result=True,
    PREFIX="1001",
    fun_evals=inf,
    fun_repeats=2,
    max_time=1,
    data_full_train = train_dataset,
    data_val=test_dataset,
    data_test=test_dataset,
    shuffle_train=False,
    shuffle_val=False,    
    core_model_name="light.regression.ManyToManyRNNRegressor",
    hyperdict=LightHyperDict,
    log_level=50,
    _L_in=1,
    _L_out=1)

# set_hyperparameter(fun_control, "optimizer", [ "Adadelta", "Adam", "Adamax"])
set_hyperparameter(fun_control, "rnn_units", [6, 12])
set_hyperparameter(fun_control, "fc_units", [6, 12])
set_hyperparameter(fun_control, "epochs", [12 , 16])
set_hyperparameter(fun_control, "batch_size", [6,12])
set_hyperparameter(fun_control, "dropout_prob", [0.0, 0.025])
set_hyperparameter(fun_control, "patience", [5,10])
set_hyperparameter(fun_control, "lr_mult", [0.1, 20.0])

design_control = design_control_init(init_size=20, repeats=2)

surrogate_control = surrogate_control_init(log_level=50, noise=True)



fun = HyperLight().fun

spot_tuner = Spot(fun=fun,fun_control=fun_control, design_control=design_control, surrogate_control=surrogate_control)

Seed set to 123
Seed set to 123


Moving TENSORBOARD_PATH: runs/ to TENSORBOARD_PATH_OLD: runs_OLD/runs_2025_01_27_19_26_44_0
Created spot_tensorboard_path: runs/spot_logs/1001_bartz11_2025-01-27_19-26-44 for SummaryWriter()
module_name: light
submodule_name: regression
model_name: ManyToManyRNNRegressor
_init_spot_writer(): Created spot_tensorboard_path: runs/spot_logs/1001_bartz11_2025-01-27_19-26-44 for SummaryWriter()


In [ ]:
#| label: 1001_user_data_run
res = spot_tuner.run()

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name   | Type          | Params | Mode  | In sizes              | Out sizes   
----------------------------------------------------------------------------------------
0 | layers | ManyToManyRNN | 66.6 K | train | [[512, 10, 1], [512]] | [512, 10, 1]
----------------------------------------------------------------------------------------
66.6 K    Trainable params
0         Non-trainable params
66.6 K    Total params
0.266     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode



In fun(): config:
{'act_fn': ELU(),
 'batch_size': 512,
 'dropout_prob': 0.01884581875246618,
 'epochs': 32768,
 'fc_units': 128,
 'lr_mult': 2.809808474191879,
 'optimizer': 'RMSprop',
 'patience': 64,
 'rnn_units': 128}
train_size: 8, val_size: 1 used for train & val data.
LightDataModule.val_dataloader(). Val. set size: 1
LightDataModule.train_dataloader(). data_train size: 8
LightDataModule.val_dataloader(). Val. set size: 1


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         hp_metric         │            nan            │
│         val_loss          │            nan            │
└───────────────────────────┴───────────────────────────┘

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name   | Type          | Params | Mode  | In sizes              | Out sizes   
----------------------------------------------------------------------------------------
0 | layers | ManyToManyRNN | 66.6 K | train | [[512, 10, 1], [512]] | [512, 10, 1]
----------------------------------------------------------------------------------------
66.6 K    Trainable params
0         Non-trainable params
66.6 K    Total params
0.266     Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


train_model result: {'val_loss': nan, 'hp_metric': nan}

In fun(): config:
{'act_fn': ELU(),
 'batch_size': 512,
 'dropout_prob': 0.01884581875246618,
 'epochs': 32768,
 'fc_units': 128,
 'lr_mult': 2.809808474191879,
 'optimizer': 'RMSprop',
 'patience': 64,
 'rnn_units': 128}
train_size: 8, val_size: 1 used for train & val data.
LightDataModule.val_dataloader(). Val. set size: 1
LightDataModule.train_dataloader(). data_train size: 8
LightDataModule.val_dataloader(). Val. set size: 1


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         hp_metric         │    0.2419612854719162     │
│         val_loss          │    0.2419612854719162     │
└───────────────────────────┴───────────────────────────┘

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


train_model result: {'val_loss': 0.2419612854719162, 'hp_metric': 0.2419612854719162}

In fun(): config:
{'act_fn': Tanh(),
 'batch_size': 4096,
 'dropout_prob': 0.009678046868863095,
 'epochs': 16384,
 'fc_units': 2048,
 'lr_mult': 13.031834796253152,
 'optimizer': 'SGD',
 'patience': 32,
 'rnn_units': 4096}
train_size: 8, val_size: 1 used for train & val data.



  | Name   | Type          | Params | Mode  | In sizes                | Out sizes    
-------------------------------------------------------------------------------------------
0 | layers | ManyToManyRNN | 50.4 M | train | [[4096, 10, 1], [4096]] | [4096, 10, 1]
-------------------------------------------------------------------------------------------
50.4 M    Trainable params
0         Non-trainable params
50.4 M    Total params
201.441   Total estimated model params size (MB)
6         Modules in train mode
0         Modules in eval mode


LightDataModule.val_dataloader(). Val. set size: 1
LightDataModule.train_dataloader(). data_train size: 8


In [ ]:
print_res_table(spot_tuner)
spot_tuner.plot_important_hyperparameter_contour(max_imp=3)

In [ ]:
len(spot_tuner.y)

## Summary

This section showed how to use user-specified data sets for the hyperparameter tuning process with `spotpython`. The user needs to provide the data set and load it as a `spotpython` `CSVDataset()` class.